In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("synthetic_banking_conversations_2000.csv")

In [4]:
data.head(1)

,post_id,platform,bank_name,post_text,category,subcategory,sentiment_label,sentiment_score,date,user_id,...,language,country,is_verified,topic_tags,extracted_entities,urgency_flag,complaint_flag,resolved_flag,response_time_hours,source_url
0,P000001,Reddit,First National Bank,Account flagged for suspicious activity; suppo...,Security & Fraud,Unauthorized Transaction,Positive,0.398,2024-11-23,U81482,...,en,AU,True,security,First National Bank,False,False,NaN,NaN,https://example.com/post/1


In [5]:
text_data = data[['post_id', 'platform', 'post_text']]

In [6]:
text_data.head(2)

,post_id,platform,post_text
0,P000001,Reddit,Account flagged for suspicious activity; suppo...
1,P000002,Yelp,Session keeps timing out while I complete form...


In [7]:
from transformers import pipeline

# Load a pretrained sentiment model
sentiment_pipeline = pipeline("sentiment-analysis")

/Users/yasminkabir/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/yasminkabir/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


In [8]:
examples = [
    "Customer support was very helpful and resolved my issue within an hour.",
    "App login fails after the latest update, can't access my account.",
    "Great promotional offer for new credit cards this month!"
]

for text in examples:
    print(text, "→", sentiment_pipeline(text))


Customer support was very helpful and resolved my issue within an hour. → [{'label': 'POSITIVE', 'score': 0.9799596071243286}]
App login fails after the latest update, can't access my account. → [{'label': 'NEGATIVE', 'score': 0.9997230172157288}]
Great promotional offer for new credit cards this month! → [{'label': 'POSITIVE', 'score': 0.9989493489265442}]


In [10]:
import pandas as pd
from tqdm import tqdm

tqdm.pandas()  # shows progress bar

text_data["sentiment"] = text_data["post_text"].progress_apply(lambda x: sentiment_pipeline(x)[0]["label"])
text_data["sentiment_score"] = text_data["post_text"].progress_apply(lambda x: sentiment_pipeline(x)[0]["score"])


  0%|          | 0/2000 [00:00<?, ?it/s]

100%|██████████| 2000/2000 [00:29<00:00, 67.35it/s]
/var/folders/87/nc2fdrmx5tlf8txjd6c8tppw0000gn/T/ipykernel_13860/1762301280.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_data["sentiment"] = text_data["post_text"].progress_apply(lambda x: sentiment_pipeline(x)[0]["label"])
100%|██████████| 2000/2000 [00:27<00:00, 72.75it/s]
/var/folders/87/nc2fdrmx5tlf8txjd6c8tppw0000gn/T/ipykernel_13860/1762301280.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_data["sentiment_score"] = text_data["po

In [11]:
text_data.head(3)

,post_id,platform,post_text,sentiment,sentiment_score
0,P000001,Reddit,Account flagged for suspicious activity; suppo...,POSITIVE,0.997109
1,P000002,Yelp,Session keeps timing out while I complete form...,POSITIVE,0.584901
2,P000003,Reddit,"Found the ATM network reliable in my city, no ...",POSITIVE,0.991860


In [14]:
text_data['sentiment'].value_counts()

sentiment
NEGATIVE    1255
POSITIVE     745
Name: count, dtype: int64